# Face Recognition
- Face recognition with LBPH and OpenCV
- Face recognition with Dlib, CNN and distance calculation


### LBPH (LOCAL BINARY PATTERNS HISTOGRAMS)
LBPH(Local Binary Patterns Histograms)은 이미지에서 지역적인 이진 패턴을 추출하여 각 픽셀의 주변 텍스처와 패턴을 특징화하는 알고리즘이다.
중앙값 픽셀을 기준으로 크거나 같으면 1 작으면 0으로 표현하고 중앙값을 제외 후 1 또는 0을 이어주면 이진수가 나타나는데 이것이 해당 픽셀이 나타내고 있는 특징으로 볼 수 있다. 이러한 이진수는 밝기를 조절할때 강력하고 각각의 이진수 패턴을 히스토그램으로 통계적으로 표현할 수 있다. 이런 히스토그램을 가지고 있는 한 픽셀을 학습한 데이터의 히스토그램과 비교하여 얼굴을 인식할 수 있다.

### Loading the dataset
- 앞으로 사용할 데이터셋의 출처 = Yale faces database
  - url: http://vision.ucsd.edu/content/yale-face-database


In [76]:
#PIL(Pillow) 이미지 처리 라이브러리(gif이미지, 간단한 이미지 작업)
from PIL import Image
import cv2 # 이미지 처리 라이브러리(컴퓨터 비전 처리)
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
# zip 압축 파일을 추출할 라이브러리
import zipfile
path = '/content/drive/MyDrive/Colab Notebooks/Project/Computer Vision/src/Datasets/yalefaces.zip'
# mode는 읽기, 쓰기, 읽고쓰기 중 어떤 작업을 할지 정하는것이다. r을 read이다.
zip_object =zipfile.ZipFile(file = path, mode = 'r')
# 현재위치에서 파일을 생성해라
zip_object.extractall('./')
zip_object.close()

### Pre-processing the images( 데이터 전처리 )
- 준비한 데이터셋을 얼굴인식 알고리즘에서 사용할 수 있도록 알맞은 형태로 전처리하는 과정


In [78]:
import os
# os.listdir()는 매개변수안에 있는 파일들을 다 보여준다
print(os.listdir('/content/yalefaces/train'))

['subject11.rightlight.gif', 'subject12.sad.gif', 'subject07.rightlight.gif', 'subject14.glasses.gif', 'subject15.glasses.gif', 'subject05.rightlight.gif', 'subject04.happy.gif', 'subject11.centerlight.gif', 'subject15.normal.gif', 'subject01.surprised.gif', 'subject02.normal.gif', 'subject13.glasses.gif', 'subject05.glasses.gif', 'subject02.rightlight.gif', 'subject11.leftlight.gif', 'subject11.noglasses.gif', 'subject02.wink.gif', 'subject06.sleepy.gif', 'subject08.sad.gif', 'subject03.sad.gif', 'subject14.rightlight.gif', 'subject08.centerlight.gif', 'subject01.leftlight.gif', 'subject07.glasses.gif', 'subject14.centerlight.gif', 'subject05.normal.gif', 'subject06.rightlight.gif', 'subject12.centerlight.gif', 'subject05.leftlight.gif', 'subject14.wink.gif', 'subject07.sleepy.gif', 'subject14.noglasses.gif', 'subject07.centerlight.gif', 'subject04.sleepy.gif', 'subject06.noglasses.gif', 'subject13.leftlight.gif', 'subject14.sleepy.gif', 'subject03.centerlight.gif', 'subject12.surpris

In [79]:
# 이미지를 가져온 다음 본 알고리즘에 맞는 형식으로 전송하는 함수
def get_image_data():
  paths=[os.path.join('/content/yalefaces/train',f) for f in os.listdir('/content/yalefaces/train')]
  print(paths)
  # 불러온 이미지 저장
  faces = []
  # 이미지의 객체 대상 저장
  ids = []
  for path in paths:
    # gif이미지를 가져와서 numpy로 변환, convert의 L mode는 단일 채널 이미지로 흑백 이미지이다.
    image = Image.open(path).convert('L')
    # PIL의 image 객체로 반환된 데이터를 opencv로 사용할려면 numpy로 변환해야된다.(unit8은 각 픽셀이 정수형 값을 갖는 이미지이다.)
    image_np = np. array(image, 'uint8')
    # 각각의 이미지의 객체 번호를 받아오는 작업
    id = int(os.path.split(path)[1].split('.')[0].replace('subject',''))
    ids.append(id)
    faces.append(image_np)
  return np.array(ids), faces


In [80]:
ids, faces = get_image_data()

['/content/yalefaces/train/subject11.rightlight.gif', '/content/yalefaces/train/subject12.sad.gif', '/content/yalefaces/train/subject07.rightlight.gif', '/content/yalefaces/train/subject14.glasses.gif', '/content/yalefaces/train/subject15.glasses.gif', '/content/yalefaces/train/subject05.rightlight.gif', '/content/yalefaces/train/subject04.happy.gif', '/content/yalefaces/train/subject11.centerlight.gif', '/content/yalefaces/train/subject15.normal.gif', '/content/yalefaces/train/subject01.surprised.gif', '/content/yalefaces/train/subject02.normal.gif', '/content/yalefaces/train/subject13.glasses.gif', '/content/yalefaces/train/subject05.glasses.gif', '/content/yalefaces/train/subject02.rightlight.gif', '/content/yalefaces/train/subject11.leftlight.gif', '/content/yalefaces/train/subject11.noglasses.gif', '/content/yalefaces/train/subject02.wink.gif', '/content/yalefaces/train/subject06.sleepy.gif', '/content/yalefaces/train/subject08.sad.gif', '/content/yalefaces/train/subject03.sad.gif

### Model Training( 모델 학습 )
- 전처리된 데이터를 활용하여 모델을 학습하는 과정

### LBPH 매개변수(하이퍼파라미터)
- Radius: LBPH는 중앙값을 중심으로 일정 반지름의 길이만큼 원을 그려 이진수(이미지의 특징)를 파악한다. 그때 반지름의 길이를 설정할 수 있는 매개변수가 Radius이다
  - 기본값: 1
- Neighbors: Radius가 범위를 정하는거라면 Neighbors는 그 범위 안에서 사용하는 픽셀의 개수를 지정한다. 이 픽셀들의 이진 패턴을 사용하여 중앙 픽셀의 특징을 표현한다.
  - 기본값: 8
- grid_x, grid_y : 히스토그램을 가질수있는 구역(특징을 가진)을 gird_x * grid_y 크기로 나눌 수 있다.
  - 기본값 :8,8
- Threshold : 신뢰도를 뜻하고 이 값보다 낮은 신뢰도의 결과를 무시하는 매개변수이다. 클수록 확실한 특징만 알 수 있다.
  - 기본값 : 1.797...

In [82]:
# threshold : 1.797...
# radius : 1
# neighbors : 8
# grid_x : 8
# grid_y : 8
lbph_classifier = cv2.face.LBPHFaceRecognizer_create(radius=4, neighbors=14, grid_x=9, grid_y=9)
lbph_classifier.train(faces, ids)
# 분류기를 저장할때 확장자는 기본적으로 yml이다.
lbph_classifier.write('lbph_classifier.yml')

### model inference ( 모델 사용 )
- 학습이 완료된 모델을 이용하여 실제 사용할 데이터의 클래스를 분류하는 과정

In [ ]:
lbph_classifier = cv2.face.LBPHFaceRecognizer_create()
# 이전에 생성한 모델을 생성한 객체에 저장
lbph_classifier.read('lbph_classifier.yml')

In [ ]:
# test데이터도 모델에 사용할 수 있도록 전처리
test_image = '/content/yalefaces/test/subject05.sleepy.gif'
image = Image.open(test_image).convert('L')
image_np = np.array(image, 'uint8')
# 모델을 사용하여 분류(분류된 클레스, 신뢰도 - 높을수록좋음)
prediction = lbph_classifier.predict(image_np)
print(prediction)

In [ ]:
# 실제 정답 클래스 추출
expected_output = int(os.path.split(test_image)[1].split('.')[0].replace('subject',''))

In [ ]:
cv2.putText(image_np, 'Pred: '+ str(prediction[0]), (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0)) # 예측
cv2.putText(image_np, 'Exp: '+ str(expected_output), (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0)) # 정답
cv2_imshow(image_np)

### Model Evaluation ( 모델 평가 )
- 학습이 완료된 모델을 이용하여 test 데이터를 사용하여 평가하는 작업


In [ ]:
paths=[os.path.join('/content/yalefaces/test',f) for f in os.listdir('/content/yalefaces/test')]
predictions =[]
expected_outuputs =[]
for path in paths:
  image = Image.open(path).convert('L')
  image_np = np. array(image, 'uint8')
  prediction = lbph_classifier.predict(image_np)
  expected_outupt = int(os.path.split(path)[1].split('.')[0].replace('subject',''))
  predictions.append(prediction[0])
  expected_outuputs.append(expected_outupt)

In [ ]:
expected_outuputs = np.array(expected_outuputs)
predictions = np.array(predictions)
print(predictions)
print(expected_outuputs)

In [ ]:
# sklearn은 대표적인 머신러닝 라이브러리이다.
from sklearn.metrics import accuracy_score
# 정확도 측정
accuracy_score(expected_outuputs, predictions)

##### 정확도 0.66666... 로 높지않다. 클래스가 전부 참이기도하고 오차행렬도 없어 정확도가 낮다.

In [ ]:
# confusion_matrix 생성
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_outuputs, predictions)
cm

In [ ]:
# 좀 더 보기 좋게 시각화
import seaborn
# annot속성은 표안에 수치를 글자로 표시할지 선택하는 속성이다.
# 열을 실제 정답 클래스, 행은 예측 클래스
# 대각선은 맞힌 횟수, 열 밖에 있는 숫자는 틀린횟수
seaborn.heatmap(cm, annot = True);

### Dlib - Detecting facial points

In [ ]:
import dlib
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
face_detector= dlib.get_frontal_face_detector()
# 얼굴 랜드마크(landmarks)를 예측하기 위한 객체 (탐지된 얼굴안에서 작동)
points_detector = dlib.shape_predictor('/content/drive/MyDrive/Colab Notebooks/Project/Computer Vision/src/Weights/shape_predictor_68_face_landmarks.dat')

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Project/Computer Vision/src/Images/people2.jpg')
detections = face_detector(image,1)
for face in detections:
  # points_detector(원본이미지, 얼굴이 감지된 구역)
  points = points_detector(image,face)
  for point in points.parts():
    cv2.circle(image, (point.x,point.y),2,(0,255,0),1)
  # 감지된 얼굴안에서 68개의 포인트를 찾아낸다.
  # print(len(points.parts()), points.parts())

  l, t,r,b = face.left(), face.top(), face.right(), face.bottom()
  cv2. rectangle(image, (l,t),(r,b),(0,255,255),3)
cv2_imshow(image)

### Detecting facial descriptors
- 각각의 얼굴 사진의 특징을 추출(CNN)


In [ ]:
import os

In [ ]:
# Resnet: https://arxiv.org/abs/1512.03385
face_detector= dlib.get_frontal_face_detector()
points_detector = dlib.shape_predictor('/content/drive/MyDrive/Colab Notebooks/Project/Computer Vision/src/Weights/shape_predictor_68_face_landmarks.dat')
# Resnet(CNN)을 이용한 얼굴 기술자 탐지 모델 생성
face_descriptor_extractor = dlib.face_recognition_model_v1('/content/drive/MyDrive/Colab Notebooks/Project/Computer Vision/src/Weights/dlib_face_recognition_resnet_model_v1.dat')

In [ ]:
index = {}
idx = 0
face_descriptors =None
paths=[os.path.join('/content/yalefaces/train',f) for f in os.listdir('/content/yalefaces/train')]
for path in paths:
  #CNN은 컬러 가능
  image = Image.open(path).convert('RGB')
  image_np = np.array(image, 'uint8')
  face_detection = face_detector(image_np,1)
  for face in face_detection:
    # 감지 안되는 얼굴 있음
    if not face.is_empty():
      l, t,r,b = face.left(), face.top(), face.right(), face.bottom()
      cv2.rectangle(image_np, (l,t),(r,b),(0,255,255),3)
      points = points_detector(image_np,face)
      for point in points.parts():
        cv2.circle(image_np, (point.x,point.y),2,(0,255,0),1)

      # 특징 추출(CNN) ,128개의 특징으로 추출
      face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np,points)
      # 추출된 특징을 리스트 형식으로 변환 후 numpy로 변환
      face_descriptor = [f for f in face_descriptor]
      face_descriptor = np.asarray(face_descriptor, dtype = np.float64)
      # 1차원의 array이를 차원을 추가하여 두개의 차원으로 변경
      # 2차원으로 바꾸는 이유는 face_descripotrs로 각각의 face_descriptor을 넣어야되기때문이다.
      face_descriptor = face_descriptor[np.newaxis, : ]
      #print(face_descriptor.shape)

      if face_descriptors is None:
        face_descriptors = face_descriptor
      else:
        face_descriptors = np.concatenate((face_descriptors, face_descriptor),axis=0)

      index[idx] = path
      idx += 1
      #cv2_imshow(image_np)

In [ ]:
# numpy의 line alg 라이브러리에 norm() 함수를 사룡하면 서로다른 행렬의 특징값의 차이를 알수있다(작을수록 비슷한것)
np.linalg.norm(face_descriptors[131] - face_descriptors[131])
# numpy array자체를 넘겨버리면 모든 리스트의 값을 하나씩 적용하여 나온 결과를 리스트로 반환한다(1은 열, 0은 행)
np.linalg.norm(face_descriptors[131] - face_descriptors, axis = 1)

### test데이터로 정확도 검증


In [ ]:
threshold = 0.55 # 최소 신뢰도

predictions =[]
expected_outputs =[]


paths=[os.path.join('/content/yalefaces/test',f) for f in os.listdir('/content/yalefaces/test')]
for path in paths:
  image = Image.open(path).convert('RGB')
  image_np = np.array(image, 'uint8')
  face_detection = face_detector(image_np,1)
  for face in face_detection: # 실습 이미지에선 하나만 나옴(얼굴이 하나이기 때문에)
    if not face.is_empty():
      points = points_detector(image_np,face)
      face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np,points)
      face_descriptor = [f for f in face_descriptor]
      face_descriptor = np.asarray(face_descriptor, dtype = np.float64)
      face_descriptor = face_descriptor[np.newaxis, : ]

      # 이전에 생성했던 face_descriptors와 비교
      distances = np.linalg.norm(face_descriptor - face_descriptors, axis = 1)
      # 리스트로 반환된 값중 최소값을 가진 행 번호(index)를 가져옴
      min_index = np.argmin(distances)
      min_distance= distances[min_index]
      if min_distance <= threshold:
        name_pred = int(os.path.split(index[min_index])[1].split('.')[0].replace('subject',''))
      else:
        print(min_distance)
        name_pred = -1 # 'Not identified'

    name_real = int(os.path.split(path)[1].split('.')[0].replace('subject',''))

    predictions.append(name_pred)
    expected_outputs.append(name_real)

    cv2.putText(image_np, 'Pred: '+ str(name_pred), (10,30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0)) # 예측
    cv2.putText(image_np, 'Exp: '+ str(name_real), (10,50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0)) # 정답
    cv2_imshow(image_np)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(expected_outputs,predictions)

연습 문제

구글 드라이브의 Dataset 폴더에서 jones_gabriel.zip 파일을 다운로드한다.
새로운 데이터셋으로 얼굴 인식을 실습하라.

- 이 데이터셋에는 학습용 이미지만 있기 때문에, 학습에 사용한 이미지로 평가도 해야 합니다.

- 알고리즘을 학습시키고 매개 변수를 조정하세요.